# Extract MS/MS Spectra

This script extracts MS/MS spectra from `.mzML` raw files generated by qTOF and QE instruments and output `.msp` files. Specifically MS/MS spectra were extracted from MS/MS raw data files (mzml format) via the pymzML parsing library. For each compound, intensity of each spectral fragment was normalized to the fragment with the highest intensity (set to 1000). Spectral fragments with intensities below 0.5% relative to the highest intensity fragment were filtered out. Compound metadata (e.g., InChIKey, collision energy) and fragmentation information (e.g., m/z, intensity) were reported for each compound. Example input files from each instrument are provided to demonstrate spectra extraction and reporting.

In [20]:
import glob
import numpy as np
from pathlib import Path
import pandas as pd
import pymzml

class SpectraExtraction:
    MACHINE_SETTINGS = {
        'QTOF': {
            'mz_tolerance': 0.01,
            'rt_window': 0.5,
            'collision_energies': [0, 10, 20, 40],
            'instrument_type': 'LC-ESI-QTOF',
            'instrument': 'Agilent qTOF 6545'
        },
        'QE': {
            'mz_tolerance': 0.01,
            'rt_window': 0.5,
            'collision_energies': [30],
            'instrument_type': 'LC-ESI-QFT',
            'instrument': 'Thermo Q Exactive HF'
        },
    }
    
    # Divides an array into chunks of a specific size
    def _chunk_array(self, arr, chunk_size):
        for i in range(0, len(arr), chunk_size):
            yield arr[i:i+chunk_size]


    # Check if a set of spectra matches the expected collision energies for
    # the particular machine, as well as if the rt's are within the rt window
    def _validate_spectra_chunk(self, chunk, rt):
        spectra_outside_rt = [spec for spec in chunk if not self._is_spectrum_within_rt(spec, rt)]

        if len(chunk) != len(self.collision_energies):
            print(f'Chunk is of incorrect size: {len(chunk)}')
            return False

        if len(spectra_outside_rt) > 0:
            print(f'{len(spectra_outside_rt)} outside rt window')
            return False

        spec_collision_energies = [int(spec['collision energy']) for spec in chunk]

        for idx in range(0, len(self.collision_energies)):
            if spec_collision_energies[idx] != self.collision_energies[idx]:
                print('Found unexpected collision energies')
                print(spec_collision_energies)
                return False

        return True

    # Check if a spectrum is within the mz tolerance
    def _is_spectrum_within_mz(self, spec, mz):
        if spec.ms_level != 2 or len(spec.selected_precursors) == 0:
            return False

        return (mz - self.mz_tolerance) <= spec.selected_precursors[0]['mz'] <= (mz + self.mz_tolerance)

    def _is_spectrum_within_rt(self, spec, rt):
        return (rt - self.rt_window) <= spec.scan_time_in_minutes() <= (rt + self.rt_window)

    
    # Filters and sorts the peaks for a particular spectrum
    def _process_peaks(self, spec):
        peaks = spec.peaks('centroided')

        if len(peaks) == 0:
            print('No peaks found')
            return []

        precursor_mz = spec.selected_precursors[0]['mz']

        # Exclude peaks with mz > precursor mz + mz tolerance
        peaks = peaks[ peaks[:,0] <= (precursor_mz + self.mz_tolerance) ]

        if len(peaks) == 0:
            print('No peaks found satisfying mz > precursor mz + mz tolerance')
            return []

        mz_max, ion_count_max = peaks.max(axis=0)

        # Normalize ion counts to 1000
        peaks[:,1] = peaks[:,1] / ion_count_max * 1000

        # Exclude peaks with normalized ion counts < 5
        peaks = peaks[ peaks[:,1] >= 5 ]

        # Sort ion counts in descending order
        # peaks = peaks[ peaks[:, 1].argsort()[::-1] ]

        # Sort mz in ascending order
        peaks = peaks[ peaks[:, 0].argsort()]

        return peaks


    # Writes out the spectrum metadata and peaks to the msp file
    def _write_spectrum_to_file(self, file, cpd, spec, collision_energy, row):
        peaks = self._process_peaks(spec)

        ionization_mode = 'positive' if row['mode'] == 'c18positive' or row['mode'] == 'hilicpositive' else 'negative'
        instrument_type = self.MACHINE_SETTINGS[self.machine]['instrument_type']
        instrument = self.MACHINE_SETTINGS[self.machine]['instrument']
        
        file.write(f'Name: {cpd}\n')
        file.write(f"Precursor_mz: {row['mz']}\n")
        file.write(f"Precursor_type: {row['adduct']}\n")
        file.write(f"Spectrum_type: MS2\n")
        file.write(f"Instrument_type: {instrument_type}\n")
        file.write(f"Instrument: {instrument}\n")
        file.write(f"InChIKey: {row['inchikey']}\n")
        file.write(f"SMILES: {row['canonical_smiles']}\n")
        file.write(f"Formula: {row['Molecular_formula_as_seen_in_ms']}\n")
        file.write(f"Ion_mode: {ionization_mode}\n")
        
        if self.machine == 'QTOF':
            file.write(f"Collision_energy: {collision_energy} eV\n")
        else:
            file.write(f"Collision_energy: HCD (NCE 20-30-40%)\n")
            
        file.write(f"Comments: Sonnenburg Lab MS2 Library\n")
        file.write(f"Num Peaks: {len(peaks)}\n")

        for peak in peaks:
            file.write(f"{round(peak[0], 5)},{round(peak[1]):.0f}\n")

        file.write('\n')
    
    
    # Given a directory, we find all the Excel library files inside,
    # take all the compounds in each library, and search the referenced mzML files
    # for spectra that match the expected mz and rt for each compound
    def extract(self, parent_dir, machine):
        self.machine = machine
        self.mz_tolerance = self.MACHINE_SETTINGS[machine]['mz_tolerance']
        self.rt_window = self.MACHINE_SETTINGS[machine]['rt_window']
        self.collision_energies = self.MACHINE_SETTINGS[machine]['collision_energies']

        print(f"Selected machine {machine} with mz tolerance {self.mz_tolerance} and rt window {self.rt_window}")

        analysis_results = []

        output_msp_file_name = f'output_{machine}.msp'
        output_msp_file = open(output_msp_file_name, 'w')
        
        libraries = glob.glob(f'{parent_dir}/*.xlsx')

        for library_file in libraries:
            print('---------------------------------------------------------')
            print(f'Library file: {library_file}')
            library = pd.read_excel(library_file, index_col=0)
            library = library[library['dname'].notnull()]

            # Only keep the first compound when a compound appears more than once
            filtered_library = library.drop_duplicates(['dname', 'Compound', 'pool'])

            # Remove all rows where dnames point to different compounds
            filtered_library = filtered_library.drop_duplicates(['dname', 'pool'], keep=False)

            print(filtered_library)

            dupes = library.loc[~library.index.isin(filtered_library.index)]

            # Add all the duplicates we've excluded to the analysis results
            for idx, row in dupes.iterrows():
                analysis_results.append({
                    'pool': row['pool'],
                    'library_file': library_file,
                    'compound': row['Compound'],
                    'dname': row['dname'],
                    'PubChem_CID': row['PubChem_CID'],
                    'Peak': row['Peak'],
                    'mz': row['mz'],
                    'rt': row['rt']
                })

            for pool, pool_cpds in filtered_library.groupby('pool'):
                print('-------------------------------------')
                print(f'Pool: {pool}')
                mzml_path = f'{parent_dir}/{pool}.mzML'

                if not Path(mzml_path).is_file():
                    print(f'{mzml_path} not found. Skipping...')
                    continue

                # We find and process mzML files based on pool names
                run = pymzml.run.Reader(mzml_path)

                for idx, row in pool_cpds.iterrows():
                    cpd = row['Compound']
                    mz = row['mz']
                    rt = row['rt']

                    print(f'\nSearching for {cpd} with mz = {mz} and rt = {rt}:')

                    found_spectra = list(spec for spec in run if self._is_spectrum_within_mz(spec, mz))

                    print(f'Found {len(found_spectra)} spectrum objects')

                    if len(found_spectra) == 0:
                        print('Skipping because no spectra found')

                        analysis_results.append({
                            'mzml_path': mzml_path,
                            'pool': pool,
                            'library_file': library_file,
                            'compound': cpd,
                            'dname': row['dname'],
                            'PubChem_CID': row['PubChem_CID'],
                            'Peak': row['Peak'],
                            'mz': mz,
                            'rt': rt,
                            'num_spectra': len(found_spectra)
                        })

                        continue

                    # We subjected each compound to a set of collision energies (depending on the machine used)
                    # and so we chunk the list of spectra here so that each chunk corresponds to a single compound
                    chunks = self._chunk_array(found_spectra, len(self.collision_energies))
                    chunks = [chunk for chunk in chunks if self._validate_spectra_chunk(chunk, rt)]

                    if len(chunks) == 0:
                        print('Skipping because all found spectra fall outside tolerance window')

                        analysis_results.append({
                            'mzml_path': mzml_path,
                            'pool': pool,
                            'library_file': library_file,
                            'compound': cpd,
                            'dname': row['dname'],
                            'PubChem_CID': row['PubChem_CID'],
                            'Peak': row['Peak'],
                            'mz': mz,
                            'rt': rt,
                            'num_spectra': len(found_spectra),
                            'all_spectra_outside_window': 'yes'
                        })

                        continue

                    # Sort chunks of spectra by the TIC of the first spectra in each chunk
                    chunks = sorted(chunks, key=lambda chunk: chunk[0].TIC, reverse=True)

                    print(f"Writing set with TIC {chunks[0][0].TIC} to output file")

                    analysis_results.append({
                        'mode': row['mode'],
                        'compound': cpd,
                        'dname': row['dname'],
                        'mz': mz,
                        'measured_rt': [spec.scan_time_in_minutes() for spec in chunks[0]][0],
                        'adduct': row['adduct'],
                        'Peak': row['Peak'],
                        'PubChem_CID': row['PubChem_CID'],
                        'Molecular_formula_as_seen_in_ms': row['Molecular_formula_as_seen_in_ms'],
                        'Monoisotopic_mass_as_seen_in_ms': row['Monoisotopic_mass_as_seen_in_ms'],
                        'canonical_smiles': row['canonical_smiles'],
                        'inchikey': row['inchikey'],
                        'num_spectra': len(chunks) * len(self.collision_energies),
                        'TIC': chunks[0][0].TIC,
                        'mzml_path': mzml_path,
                        'pool': pool,
                        'library_file': library_file,
                    })

                    for idx, spec in enumerate(chunks[0]):
                        # Skip collision energy 0
                        if self.collision_energies[idx] == 0:
                            continue

                        self._write_spectrum_to_file(output_msp_file, cpd, spec, self.collision_energies[idx], row)

        output_msp_file.close()

        print(f'\nCreated output file: {output_msp_file_name}')

        analysis_results = pd.DataFrame(analysis_results)
        analysis_results.to_excel(f'extract_ms2_spectra_results.xlsx')
        
        return analysis_results

In [21]:
qe_extraction = SpectraExtraction()
qe_analysis_results = qe_extraction.extract('input/QE_example', 'QE')

Selected machine QE with mz tolerance 0.01 and rt window 0.5
---------------------------------------------------------
Library file: input/QE_example/ms2_extraction_QE_database_input_example.xlsx
           mode                              Compound          mz     rt  \
1   c18negative           IS_GLUCOSE-1,2,3,4,5,6,6-D7  186.100024  0.817   
2   c18negative               IS_2-FLUROPHENYLGLYCINE  168.046624  1.205   
3   c18negative               IS_METHIONINE-METHYL-D3  151.062624  1.174   
4   c18negative                   IS_LEUCINE-5,5,5-D3  133.106224  1.548   
5   c18negative            IS_TRYPTOPHAN-2,4,5,6,7-D5  208.114024  2.517   
6   c18negative               IS_N-BENZOYL-D5-GLYCINE  183.082324  2.333   
7   c18negative             IS_4-CHLORO-PHENYLALANINE  198.032724  3.368   
8   c18negative              IS_4-BROMO-PHENYLALANINE  241.982224  3.626   
9   c18negative  IS_INDOLE-2,4,5,6,7-D5-3-ACETIC ACID  179.087424  2.406   
10  c18negative                  IS_D15-OCTA

In [22]:
qe_analysis_results

,mode,compound,dname,mz,measured_rt,adduct,Peak,PubChem_CID,Molecular_formula_as_seen_in_ms,Monoisotopic_mass_as_seen_in_ms,canonical_smiles,inchikey,num_spectra,TIC,mzml_path,pool,library_file
0,c18negative,"IS_GLUCOSE-1,2,3,4,5,6,6-D7",m_c18n_0553,186.100024,0.782882,[M-H]-,NaN,92043367,C6H12O6,187.1073,C(C1C(C(C(C(O1)O)O)O)O)O,WQZGKKKJIJFFOK-QCQRZMBHSA-N,18,1.058966e+06,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
1,c18negative,IS_2-FLUROPHENYLGLYCINE,m_c18n_0554,168.046624,1.496549,[M-H]-,NaN,5171030,C8H8FNO2,169.0539,C1=CC=C(C(=C1)NCC(=O)O)F,VCJRLZZBUWJOGG-UHFFFAOYSA-N,58,4.830894e+04,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
2,c18negative,IS_METHIONINE-METHYL-D3,m_c18n_0555,151.062624,0.869770,[M-H]-,NaN,15556503,C5H11NO2S,152.0699,CSCCC(C(=O)O)N,FFEARJCKVFRZRR-OSIBIXDNSA-N,141,1.696431e+06,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
3,c18negative,"IS_LEUCINE-5,5,5-D3",m_c18n_0556,133.106224,1.672887,[M-H]-,NaN,11073472,C6H13NO2,134.1135,CC(C)CC(C(=O)O)N,ROHFNLRQFUQHCH-LONBSJBQSA-N,123,1.629402e+07,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
4,c18negative,"IS_TRYPTOPHAN-2,4,5,6,7-D5",m_c18n_0557,208.114024,2.662946,[M-H]-,NaN,12209747,C11H12N2O2,209.1213,C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N,QIVBCDIJIAJPQS-HLTLGYGQSA-N,64,8.148265e+07,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
5,c18negative,IS_N-BENZOYL-D5-GLYCINE,m_c18n_0558,183.082324,2.417578,[M-H]-,NaN,101624378,C9H9NO3,184.0896,C1=CC=C(C=C1)C(=O)NCC(=O)O,QIAFMBKCNZACKA-RALIUCGRSA-N,73,8.550012e+07,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
6,c18negative,IS_4-CHLORO-PHENYLALANINE,m_c18n_0559,198.032724,3.529703,[M-H]-,NaN,736190,C9H10ClNO2,199.0400,C1=CC(=CC=C1CC(C(=O)O)N)Cl,NIGWMJHCCYYCSF-QMMMGPOBSA-N,78,1.134867e+08,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
7,c18negative,IS_4-BROMO-PHENYLALANINE,m_c18n_0560,241.982224,3.780225,[M-H]-,NaN,671214,C9H10BrNO2,242.9895,C1=CC(=CC=C1CC(C(=O)O)N)Br,PEMUHKUIQHFMTH-QMMMGPOBSA-N,94,3.061529e+08,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
8,c18negative,"IS_INDOLE-2,4,5,6,7-D5-3-ACETIC ACID",m_c18n_0561,179.087424,2.493227,[M-H]-,NaN,54301591,C10H9NO2,180.0947,C1=CC=C2C(=C1)C(=CN2)CC(=O)O,SEOVTRFCIGRIMH-SNOLXCFTSA-N,59,8.635401e+06,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...
9,c18negative,IS_D15-OCTANOIC ACID,m_c18n_0562,158.201924,4.673705,[M-H]-,NaN,13011402,C8H16O2,159.2092,CCCCCCCC(=O)O,WWZKQHOCKIZLMA-PMELWRBQSA-N,116,5.470386e+08,input/QE_example/new_istd.mzML,new_istd,input/QE_example/ms2_extraction_QE_database_in...


In [23]:
with open('output_QE.msp', 'r') as msp:
    print(msp.read())

Name: IS_GLUCOSE-1,2,3,4,5,6,6-D7
Precursor_mz: 186.100024
Precursor_type: [M-H]-
Spectrum_type: MS2
Instrument_type: LC-ESI-QFT
Instrument: Thermo Q Exactive HF
InChIKey: WQZGKKKJIJFFOK-QCQRZMBHSA-N
SMILES: C(C1C(C(C(C(O1)O)O)O)O)O
Formula: C6H12O6
Ion_mode: negative
Collision_energy: HCD (NCE 20-30-40%)
Comments: Sonnenburg Lab MS2 Library
Num Peaks: 22
59.0137,53
60.02008,496
61.02629,1000
68.62632,26
72.02028,245
73.02636,461
73.99946,33
74.03281,138
77.83348,23
81.56016,26
89.0546,41
90.03053,62
91.03693,272
92.04318,274
93.04964,242
104.04327,86
105.04948,130
116.04323,27
117.04961,167
124.06639,322
136.35278,22
162.89946,31

Name: IS_2-FLUROPHENYLGLYCINE
Precursor_mz: 168.046624
Precursor_type: [M-H]-
Spectrum_type: MS2
Instrument_type: LC-ESI-QFT
Instrument: Thermo Q Exactive HF
InChIKey: VCJRLZZBUWJOGG-UHFFFAOYSA-N
SMILES: C1=CC=C(C(=C1)NCC(=O)O)F
Formula: C8H8FNO2
Ion_mode: negative
Collision_energy: HCD (NCE 20-30-40%)
Comments: Sonnenburg Lab MS2 Library
Num Peaks: 12
63.22

In [24]:
qtof_extraction = SpectraExtraction()
qtof_analysis_results = qtof_extraction.extract('input/qTOF_example', 'QTOF')

Selected machine QTOF with mz tolerance 0.01 and rt window 0.5
---------------------------------------------------------
Library file: input/qTOF_example/ms2_extraction_qTOF_database_input_example.xlsx
           mode                              Compound          mz     rt  \
1   c18negative           IS_GLUCOSE-1,2,3,4,5,6,6-D7  186.100024  0.817   
2   c18negative               IS_2-FLUROPHENYLGLYCINE  168.046624  1.205   
3   c18negative               IS_METHIONINE-METHYL-D3  151.062624  1.174   
4   c18negative                   IS_LEUCINE-5,5,5-D3  133.106224  1.548   
5   c18negative            IS_TRYPTOPHAN-2,4,5,6,7-D5  208.114024  2.517   
6   c18negative               IS_N-BENZOYL-D5-GLYCINE  183.082324  2.333   
7   c18negative             IS_4-CHLORO-PHENYLALANINE  198.032724  3.368   
8   c18negative              IS_4-BROMO-PHENYLALANINE  241.982224  3.626   
9   c18negative  IS_INDOLE-2,4,5,6,7-D5-3-ACETIC ACID  179.087424  2.406   
10  c18negative                  IS_D1

In [25]:
qtof_analysis_results

,mode,compound,dname,mz,measured_rt,adduct,Peak,PubChem_CID,Molecular_formula_as_seen_in_ms,Monoisotopic_mass_as_seen_in_ms,canonical_smiles,inchikey,num_spectra,TIC,mzml_path,pool,library_file
0,c18negative,"IS_GLUCOSE-1,2,3,4,5,6,6-D7",m_c18n_0553,186.100024,0.842883,[M-H]-,NaN,92043367,C6H12O6,187.1073,C(C1C(C(C(C(O1)O)O)O)O)O,WQZGKKKJIJFFOK-QCQRZMBHSA-N,16,7.409802e+04,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
1,c18negative,IS_2-FLUROPHENYLGLYCINE,m_c18n_0554,168.046624,1.265500,[M-H]-,NaN,5171030,C8H8FNO2,169.0539,C1=CC=C(C(=C1)NCC(=O)O)F,VCJRLZZBUWJOGG-UHFFFAOYSA-N,12,1.249359e+04,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
2,c18negative,IS_METHIONINE-METHYL-D3,m_c18n_0555,151.062624,1.027200,[M-H]-,NaN,15556503,C5H11NO2S,152.0699,CSCCC(C(=O)O)N,FFEARJCKVFRZRR-OSIBIXDNSA-N,24,2.313995e+05,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
3,c18negative,"IS_LEUCINE-5,5,5-D3",m_c18n_0556,133.106224,1.621500,[M-H]-,NaN,11073472,C6H13NO2,134.1135,CC(C)CC(C(=O)O)N,ROHFNLRQFUQHCH-LONBSJBQSA-N,52,5.895498e+05,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
4,c18negative,"IS_TRYPTOPHAN-2,4,5,6,7-D5",m_c18n_0557,208.114024,2.635067,[M-H]-,NaN,12209747,C11H12N2O2,209.1213,C1=CC=C2C(=C1)C(=CN2)CC(C(=O)O)N,QIVBCDIJIAJPQS-HLTLGYGQSA-N,36,1.336541e+06,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
5,c18negative,IS_N-BENZOYL-D5-GLYCINE,m_c18n_0558,183.082324,2.477150,[M-H]-,NaN,101624378,C9H9NO3,184.0896,C1=CC=C(C=C1)C(=O)NCC(=O)O,QIAFMBKCNZACKA-RALIUCGRSA-N,60,1.407208e+06,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
6,c18negative,IS_4-CHLORO-PHENYLALANINE,m_c18n_0559,198.032724,3.528433,[M-H]-,NaN,736190,C9H10ClNO2,199.0400,C1=CC(=CC=C1CC(C(=O)O)N)Cl,NIGWMJHCCYYCSF-QMMMGPOBSA-N,24,9.860740e+05,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
7,c18negative,IS_4-BROMO-PHENYLALANINE,m_c18n_0560,241.982224,3.787483,[M-H]-,NaN,671214,C9H10BrNO2,242.9895,C1=CC(=CC=C1CC(C(=O)O)N)Br,PEMUHKUIQHFMTH-QMMMGPOBSA-N,28,3.078662e+06,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
8,c18negative,"IS_INDOLE-2,4,5,6,7-D5-3-ACETIC ACID",m_c18n_0561,179.087424,2.504033,[M-H]-,NaN,54301591,C10H9NO2,180.0947,C1=CC=C2C(=C1)C(=CN2)CC(=O)O,SEOVTRFCIGRIMH-SNOLXCFTSA-N,44,6.508262e+05,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...
9,c18negative,IS_D15-OCTANOIC ACID,m_c18n_0562,158.201924,4.745800,[M-H]-,NaN,13011402,C8H16O2,159.2092,CCCCCCCC(=O)O,WWZKQHOCKIZLMA-PMELWRBQSA-N,48,3.550485e+06,input/qTOF_example/new_istd.mzML,new_istd,input/qTOF_example/ms2_extraction_qTOF_databas...


In [26]:
with open('output_QTOF.msp', 'r') as msp:
    print(msp.read())

Name: IS_GLUCOSE-1,2,3,4,5,6,6-D7
Precursor_mz: 186.100024
Precursor_type: [M-H]-
Spectrum_type: MS2
Instrument_type: LC-ESI-QTOF
Instrument: Agilent qTOF 6545
InChIKey: WQZGKKKJIJFFOK-QCQRZMBHSA-N
SMILES: C(C1C(C(C(C(O1)O)O)O)O)O
Formula: C6H12O6
Ion_mode: negative
Collision_energy: 10 eV
Comments: Sonnenburg Lab MS2 Library
Num Peaks: 56
42.00693,11
46.00313,10
57.03445,11
58.00551,11
58.03674,9
59.01301,14
59.02278,9
60.02008,363
60.0303,130
61.02661,1000
61.0362,550
61.06825,18
61.07774,7
62.03271,277
62.04267,151
62.06144,8
72.0209,51
72.03076,41
73.02591,264
73.03811,81
74.03279,95
79.03615,7
88.01445,6
89.05737,7
90.03044,6
91.03717,138
91.04963,71
92.04329,230
92.05488,123
93.04961,155
93.06115,72
94.05572,27
94.075,6
97.02725,6
97.0371,7
98.0324,25
101.02493,5
101.03479,8
101.06621,10
103.03465,6
103.05001,6
104.04118,24
104.05764,12
105.04906,44
105.07045,13
106.05723,10
116.04519,17
116.06611,6
117.05029,62
118.05501,42
118.07198,13
124.06666,273
124.08019,125
125.08939,5
18